In [17]:
import pandas as pd
from urllib.parse import quote
import sqlalchemy
from sqlalchemy import create_engine
import psycopg2

# Test data 생성 as pandas #
test_data = pd.DataFrame({
        'date': ['2019-07-28']*4, 
        'name': ['kim', 'lee', 'choi', 'park'], 
        'age': [19, 20, 19, 20], 
        'math_score': [91, 95, 92, 70], 
        'pass_yn': [True, True, True, False]}, 
         columns=['date', 'name', 'age', 'math_score', 'pass_yn'])

# DB ORM 연결, 패스워드 특수문자 치환 #
engine = create_engine('postgresql://app_fs:%s@localhost:5432/db_fs' % quote('pok1234@'),echo = True) 
engine.execute("DROP TABLE IF EXISTS public.tb_test;") # drop table if exists
# data to DBMS Table #
test_data.to_sql(
         name = 'tb_test',
         con = engine,
         schema = 'public',
         if_exists = 'append',
         index = False
         )

engine.dispose()

#Postres 연결 #
connection = psycopg2.connect(database="db_fs", user="app_fs", password="pok1234@")
cursor = connection.cursor()
sql = "select * from tb_test;"
cursor.execute(sql)
ts = cursor.fetchall()
ts
cursor.close()
connection.close()

2022-02-19 21:46:10,431 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2022-02-19 21:46:10,432 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-02-19 21:46:10,433 INFO sqlalchemy.engine.Engine select current_schema()
2022-02-19 21:46:10,433 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-02-19 21:46:10,434 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2022-02-19 21:46:10,434 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-02-19 21:46:10,434 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS public.tb_test;
2022-02-19 21:46:10,435 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-02-19 21:46:10,436 INFO sqlalchemy.engine.Engine COMMIT
2022-02-19 21:46:10,450 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where n.nspname=%(schema)s and relname=%(name)s
2022-02-19 21:46:10,450 INFO sqlalchemy.engine.Engine [generated in 0.00045s] {'schema': 'public', 'name': 'tb_test'}
2022-02-19 21:46:10,452 INFO sqlalchem

In [19]:
import os
import sys
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
import psycopg2
from sqlalchemy import null

## Function ## match result 유효한 데이터 값만 가져오기
def gt_matchResult(st_data):
    rows = list()
    for x in range(3,30):
        if st_data[x][2] == '-':
            break
        extend_row = list(st_data[x][0:15])
        rows.append(extend_row)
    return rows

def gt_teamResult(st_data):
    rows = list()
    for x in range(3,30):
        if st_data[x][20] == '':
            break
        else:
            if st_data[x][21] == '':
                pass
            else:
                extend_row = list(st_data[x][20:22])
                rows.append(extend_row)
    return rows    

## Function ## Match Data Insert
def Insert_gameresult(game_data):
    sql = "INSERT INTO tb_gameresult (season) VALUES(%s)"
    cursor.execute(sql, 2021)

## Function ## Player and Team
# def Insert_playerpick(team_result):
#     pick_data = game_data


# Google API 연결 설정 #
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive',]
work_path = os.getcwd()
resource_path = os.path.join(work_path,'resources')
json_file_name = os.path.join(resource_path,'quiet-amp-275114-083a1f3b2f00.json')
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
gc = gspread.authorize(credentials)

# Postres 연결 #
connection = psycopg2.connect(database="db_fs", user="app_fs", password="pok1234@")
cursor = connection.cursor()

# URL List 가져오기 #
sql = "select * from tb_sheeturls"
cursor.execute(sql)
url_data = cursor.fetchall()
print("List of URLS updated")

# 구글시트 url 리스트 #
spreadsheet_url = url_data[0][0] # 1경기 URL
match_NumAndDate = url_data[0][1]
match_Num = match_NumAndDate[:3]
match_Date = match_NumAndDate[-8:]

# 스프레스시트 문서 가져오기
doc = gc.open_by_url(spreadsheet_url)
worksheet = doc.sheet1  ## 시트 선택하기
sheet_data = worksheet.get_all_values()  ## 시트 전체 값을 List of List 로 저장

# 3번째 행을 header로 지정
#header = sheet_data[2][20:22]

# Team Select 결과 #
team_result = list(sheet_data[20:22])

#pd_rows = gt_matchResult(sheet_data)
pd_rows = gt_teamResult(sheet_data)

# Pandas 출력
matchdata = pd.DataFrame(pd_rows, columns= ["team","name"])
matchdata['mNum'] = match_Num
matchdata['mDate'] = match_Date
print(matchdata)

# DB ORM 연결, 패스워드 특수문자 치환 #
engine = create_engine('postgresql://app_fs:%s@localhost:5432/db_fs' % quote('pok1234@'),echo = True) 
#engine.execute("DROP TABLE IF EXISTS public.tb_test;") # drop table if exists
# data to DBMS Table #
matchdata.to_sql(
         name = 'tb_test',
         con = engine,
         schema = 'public',
         if_exists = 'append',
         index = False,
        #  dtype = {
        #         'team': sqlalchemy.types.VARCHAR(),
        #         'name': sqlalchemy.types.VARCHAR()
        #         }
        )
engine.dispose()


List of URLS updated
2022-02-19 21:47:58,224 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2022-02-19 21:47:58,225 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-02-19 21:47:58,226 INFO sqlalchemy.engine.Engine select current_schema()
2022-02-19 21:47:58,227 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-02-19 21:47:58,228 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2022-02-19 21:47:58,229 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-02-19 21:47:58,231 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where n.nspname=%(schema)s and relname=%(name)s
2022-02-19 21:47:58,231 INFO sqlalchemy.engine.Engine [generated in 0.00055s] {'schema': 'public', 'name': 'tb_test'}
2022-02-19 21:47:58,234 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-19 21:47:58,235 INFO sqlalchemy.engine.Engine INSERT INTO public.tb_test (team, name) VALUES (%(team)s, %(name)s)
2022-02-19 21:47:58,235 INFO sqlalchemy.eng

In [24]:
import os
import sys
import datetime
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
import psycopg2
from sqlalchemy import null

## Function ## match result 유효한 데이터 값만 가져오기
def gt_matchResult(st_data):
    rows = list()
    for x in range(3,30):
        if st_data[x][2] == '-':
            break
        extend_row = list(st_data[x][0:15])
        rows.append(extend_row)
    return rows

def gt_teamResult(st_data):
    rows = list()
    for x in range(3,30):
        if st_data[x][20] == '':
            break
        else:
            if st_data[x][21] == '':
                pass
            else:
                extend_row = list(st_data[x][20:22])
                rows.append(extend_row)
    return rows    

## Function ## Match Data Insert
def Insert_gameresult(game_data):
    sql = "INSERT INTO tb_gameresult (season) VALUES(%s)"
    cursor.execute(sql, 2021)

## Function ## Player and Team
# def Insert_playerpick(team_result):
#     pick_data = game_data

# Google API 연결 설정 #
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive',]
work_path = os.getcwd()
resource_path = os.path.join(work_path,'resources')
json_file_name = os.path.join(resource_path,'quiet-amp-275114-083a1f3b2f00.json')
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
gc = gspread.authorize(credentials)

# Postres 연결 #
connection = psycopg2.connect(database="db_fs", user="app_fs", password="pok1234@")
cursor = connection.cursor()

# URL List 가져오기 #
sql = "select * from tb_sheeturls"
cursor.execute(sql)
url_data = cursor.fetchall()
print("List of URLS updated")

# 구글시트 url 리스트 #
spreadsheet_url = url_data[0][0] # 1경기 URL
match_NumAndDate = url_data[0][1]
match_Num = match_NumAndDate[:3]
match_Date = match_NumAndDate[-8:]

# 스프레스시트 문서 가져오기
doc = gc.open_by_url(spreadsheet_url)
worksheet = doc.sheet1  ## 시트 선택하기
sheet_data = worksheet.get_all_values()  ## 시트 전체 값을 List of List 로 저장

# 3번째 행을 header로 지정
#header = sheet_data[2][20:22]

# Team Select 결과 #
team_result = list(sheet_data[20:22])

#pd_rows = gt_matchResult(sheet_data)
pd_rows = gt_teamResult(sheet_data)

# Pandas 출력
matchdata = pd.DataFrame(pd_rows, columns= ["team","name"])
matchdata['mNum'] = match_Num
matchdata['mDate'] = match_Date
format = '%Y-%m-%d'
convert_date = datetime.datetime.strftime(match_Date, "%Y%m%d").date()
print(convert_date)


List of URLS updated


TypeError: descriptor 'strftime' for 'datetime.date' objects doesn't apply to a 'str' object